In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [23]:
df=pd.read_csv("cardekho_imputated.csv",index_col=[0])

In [24]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [25]:
#Data Cleaning
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [26]:
df.drop('car_name',axis=1,inplace=True)
df.drop('brand',axis=1,inplace=True)

In [27]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [28]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [29]:
num_features=[feature for feature in df.columns if df[feature].dtype!='O' ]
print('Num of Numerical Features:',len(num_features))
cat_features=[feature for feature in df.columns if df[feature].dtype=='O']
print('Num of Categorical Features:',len(cat_features))
discrete_features=[feature for feature in num_features if len(df[feature].unique())<=25]
print('Num of Discrete Features:',len(discrete_features))
continuous_features=[feature for feature in num_features if feature not in discrete_features]
print('Num of continuous Features:',len(continuous_features))

Num of Numerical Features: 7
Num of Categorical Features: 4
Num of Discrete Features: 2
Num of continuous Features: 5


In [30]:
from sklearn.model_selection import train_test_split
X=df.drop(['selling_price'],axis=1)
y=df['selling_price']

In [31]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [32]:
#Feature Encoding and Scaling
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['model']=le.fit_transform(X['model'])

In [33]:
num_features=X.select_dtypes(exclude="object").columns
onehot_columns=['seller_type','fuel_type','transmission_type'] #ohe here as variety is less
label_encoder_columns=['model'] #use labelencoder here because we have a long list of variety of models

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder(drop='first')

preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehot_columns),
        ("StandardScaler",numeric_transformer,num_features)
        
    ],remainder='passthrough' #only categorical features will change the rest will remain the same
)


In [34]:
X=preprocessor.fit_transform(X)

In [35]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=49)

In [36]:
X_train

array([[ 1.        ,  0.        ,  0.        , ..., -1.32425883,
        -1.26335238, -0.40302241],
       [ 0.        ,  0.        ,  0.        , ..., -0.55279869,
        -0.32319622, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  0.21482334,
         0.07008692, -0.40302241],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.34414029, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55087963,
        -0.27525757, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  1.32979434,
         0.91715831,  2.07344426]])

In [37]:
#Model Training
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [38]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae, rmse, r2_square

In [39]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
}

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    model_train_mae, model_train_rmse, model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2=evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])

    print("Model performance for Training Set")
    print("-Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("-R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------------')

    print("Model performance for Test Set")
    print("-Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("-R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

Linear Regression
Model performance for Training Set
-Root Mean Squared Error: 492823.3287
-Mean Absolute Error: 267098.3341
-R2 Score: 0.6710
----------------------------------------
Model performance for Test Set
-Root Mean Squared Error: 712417.5138
-Mean Absolute Error: 265406.9572
-R2 Score: 0.5139


Lasso
Model performance for Training Set
-Root Mean Squared Error: 492823.3327
-Mean Absolute Error: 267096.6548
-R2 Score: 0.6710
----------------------------------------
Model performance for Test Set
-Root Mean Squared Error: 712417.0847
-Mean Absolute Error: 265403.9081
-R2 Score: 0.5139


Ridge
Model performance for Training Set
-Root Mean Squared Error: 492823.9740
-Mean Absolute Error: 267066.2338
-R2 Score: 0.6710
----------------------------------------
Model performance for Test Set
-Root Mean Squared Error: 712420.4070
-Mean Absolute Error: 265343.0868
-R2 Score: 0.5139


K-Neighbors Regressor
Model performance for Training Set
-Root Mean Squared Error: 234501.0046
-Mean Ab

In [40]:
#Hyperparameter Tuning
knn_params={"n_neighbors":[2,3,10,20,40,50]}
rf_params={"max_depth":[5,8,25,None,10],
           "max_features":[5,7,"auto",8],
           "min_samples_split":[2,8,15,20],
           "n_estimators":[100,200,500,1000]}

In [41]:
randomcv_models=[('KNN',KNeighborsRegressor(),knn_params),
                 ('RF',RandomForestRegressor(),rf_params)]

In [42]:
from sklearn.model_selection import RandomizedSearchCV
model_param={}
for name,model, params in randomcv_models:
    random=RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"-------------------------Best Params for {model_name}----------------------------")
    print(model_param[model_name])


Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
-------------------------Best Params for KNN----------------------------
{'n_neighbors': 2}
-------------------------Best Params for RF----------------------------
{'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}


In [45]:
models={
    "Random Forest Regressor":RandomForestRegressor(n_estimators= 1000, min_samples_split= 2, max_features= 8, max_depth= None),
    "K-Neighbors Regressor":KNeighborsRegressor(n_neighbors=10, n_jobs=-1)
}

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    model_train_mae, model_train_rmse, model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2=evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])

    print("Model performance for Training Set")
    print("-Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("-R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------------')

    print("Model performance for Test Set")
    print("-Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("-R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

Random Forest Regressor
Model performance for Training Set
-Root Mean Squared Error: 89716.8785
-Mean Absolute Error: 38490.6034
-R2 Score: 0.9891
----------------------------------------
Model performance for Test Set
-Root Mean Squared Error: 509633.7523
-Mean Absolute Error: 101454.0414
-R2 Score: 0.7512


K-Neighbors Regressor
Model performance for Training Set
-Root Mean Squared Error: 281202.1059
-Mean Absolute Error: 103288.5839
-R2 Score: 0.8929
----------------------------------------
Model performance for Test Set
-Root Mean Squared Error: 601775.9147
-Mean Absolute Error: 117695.7103
-R2 Score: 0.6531


